In [1]:
import importlib
from utils.tokenization import *


In [2]:
import pandas as pd
import numpy as np
from sklearn import metrics
from tqdm import tqdm_notebook
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
# import vocab
from collections import Counter
import torch.nn.functional as F
from utils import general_training
import datetime
from time import gmtime, strftime

In [3]:
from utils.checkpointing import Checkpoint

In [4]:
import bpemb


In [5]:
vocab_max_size = 50000

In [6]:
bpe_ru = bpemb.BPEmb(lang='ru', vs=vocab_max_size, dim=300)

In [7]:
lenta_df = pd.read_csv('lenta_gpt_dataset.csv')

In [8]:
lenta_df.head(4)

,url,title,text,topic,tags,split
0,https://lenta.ru/news/2006/05/09/detective/,Неизвестный расстрелял полицейский участок в о...,В понедельник неизвестный мужчина ворвался в з...,Мир,Все,train
1,https://lenta.ru/news/2005/08/18/hearing/,Адвокат спас Майкла Джексона от ареста,Адвокат Майкла Джексона приехал на заседание с...,Культура,Все,train
2,https://lenta.ru/news/2005/12/02/elections/,"Ющенко пообещал не пронести ""ни копейки мимо з...",Президент Украины Виктор Ющенко призвал всех у...,Бывший СССР,Все,train
3,https://lenta.ru/news/2002/04/24/foreigners/,МВД начинает тотальную проверку иностранцев,Российские правоохранительные органы планируют...,Россия,Все,train


In [9]:
bpe_encoded_texts = torch.load('bpe_texts.pkl')

In [10]:
bpe_encoded_headlines = torch.load('bpe_headlines.pkl')

In [11]:
train_split, val_split, test_split = [lenta_df.split == s for s in ('train', 'val', 'test')] 

In [12]:
train_raw_texts, val_raw_texts = [lenta_df.text[spl].values for spl in (train_split, val_split)]

In [13]:
train_texts, val_texts = [bpe_encoded_texts[spl] for spl in (train_split, val_split)]
train_headlines, val_headlines = [bpe_encoded_headlines[spl] for spl in (train_split, val_split)]

In [14]:
special_symbols = ['<PAD>', '<UNK>' ,'<SEP>', '</S>', 'SEG_TEXT', 'SEG_SUMMARY']

In [15]:
PAD_TOKEN, UNK_TOKEN, SEP_TOKEN, EOS_TOKEN, SEG_TEXT, SEG_TEXT_SUMMARY = special_symbols

In [16]:
vocab = build_vocab_from_pretrained_bpe(train_texts, special_symbols, bpe_ru, vocab_max_size, 1)


OOV subwords:  [('[', 2356), (']', 2355), ('@', 2197), ('”', 439), ('>', 164), ('─', 155), ('•', 145), ('<', 137), ('è', 89), ('`', 75)]


In [17]:
print(len(vocab))

48763


In [18]:
ID_SEGMENT_TEXT, ID_SEGMENT_SUMMARY = vocab.word2id(SEG_TEXT), vocab.word2id(SEG_TEXT_SUMMARY)

In [19]:
ID_SEP = vocab.word2id(SEP_TOKEN)

In [20]:
train_text_ids, val_text_ids, train_headline_ids, val_headline_ids = [transform_bpe_to_ids(texts, vocab) for texts in
                                                                     (train_texts, val_texts, train_headlines, val_headlines)]

In [21]:
pretrained_embeddings = extract_pretrained_bpe_embeddings(vocab, bpe_ru)

### Preparing tensors

In [22]:
def build_training_example(text_ids, headline_ids, vocab, truncated_length):
    if headline_ids is None:
#     assert len(headline_ids) + 1 <= truncated_length
        free_text_space = truncated_length - 1 #
    else:
        free_text_space = truncated_length - len(headline_ids) - 2
        
    text_section_length = min(free_text_space, len(text_ids))
    example = text_ids[:text_section_length]
    
    example.append(vocab.word2id(SEP_TOKEN))
    if headline_ids is not None:
        assert len(headline_ids) + len(example) + 1 <= truncated_length
        example.extend(headline_ids)
        example.append(vocab.word2id(EOS_TOKEN))
        
    return torch.tensor(example), text_section_length

In [23]:
def build_tensors(examples, text_segment_lengths, padded_length=None):
    text_segment_lengths = torch.tensor(text_segment_lengths)
    
    tensor_length = padded_length if padded_length else max(len(example) for example in examples)
    
    batch_size = len(text_segment_lengths)
    word_ids = torch.zeros(batch_size, tensor_length, dtype=torch.long)
    
    for i in tqdm_notebook(range(batch_size)):
        word_ids[i,:examples[i].size(0)] = examples[i]
        
    segment_ids = torch.empty(batch_size, tensor_length, dtype=torch.long)
    segment_ids_mask = torch.arange(tensor_length).expand(batch_size, tensor_length) >= text_segment_lengths.view(-1,1)
    segment_ids[segment_ids_mask] = ID_SEGMENT_SUMMARY
    segment_ids[~segment_ids_mask] = ID_SEGMENT_TEXT
    
    position_ids = torch.arange(tensor_length, dtype=torch.long).repeat(batch_size, 1)
    shifted_position_ids = position_ids - text_segment_lengths.view(-1,1)
    mask = shifted_position_ids > 0
    position_ids[mask] = shifted_position_ids[mask] - 1

    return  word_ids, segment_ids, position_ids

In [24]:
def build_tensors_from_ids(text_ids, headline_ids, vocab, truncated_length):
    text_tensors = []
    text_segment_lengths = []
    for i in range(len(text_ids)):
        text = text_ids[i]
        headline = headline_ids[i] if headline_ids else None
        text_tensor, text_segment_length = build_training_example(text, headline, vocab, truncated_length)
        text_tensors.append(text_tensor)
        text_segment_lengths.append(text_segment_length)        
    
    token_tensor, segment_tensor, position_ids = build_tensors(text_tensors, text_segment_lengths, truncated_length)
    return token_tensor, segment_tensor, position_ids

In [25]:
truncated_length = 300

In [26]:
train_text_tensor, train_segment_tensor, train_positions_tensor = build_tensors_from_ids(train_text_ids, train_headline_ids, vocab, truncated_length)

In [27]:
val_text_tensor, val_segment_tensor, val_positions_tensor = build_tensors_from_ids(val_text_ids, val_headline_ids, vocab, truncated_length)

In [28]:
print(train_text_tensor[0])

tensor([    8,   419,  6452,   846, 21317,     8,  1323,  6428,  5199,     8,
         3500,  4616,   250,     8,  8049,  7515,     9,  3758,  7422,     6,
          202,    10,   200,   763,   154,   995,  2707, 18503,  2153,     7,
            8,   146,  1854,   137,  5723,    49,  2981,     6,    91,    12,
         8294,  1856,   176,   324,  4734,     6,     8,    76,   244, 19152,
         5199,     7,     8,   225,  4338,  2447,  2560,   356,     6, 32748,
           52,   319, 11204,     6,    49,  2981,     7,   281,  6292,   219,
            6,  2560,   494,    65,   854,     6,    67,  1344,    23,  6851,
         6798,  2523,   141,    17,  2654,   426, 15212,     7,   129,    17,
          470,   669,   508,  3194,    12,  6823,    66,  9397,  8963,     7,
           26,   202,     6,   846,    49,  1591,  1793,  1457, 13135,   476,
            6,  1457,  1882,  5892, 17733,  4309,     7,  2657,   759,   100,
           15,   693,    12,  2313,   331,     7,    12,  3041, 

In [29]:
print(train_positions_tensor[0])

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,   0,
          1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  

In [30]:
def make_tensors_for_sampling_ids(ids, vocab, reserved_space, max_text_length, beam_width):
    text_length = min(len(ids), max_text_length)
    ids = ids[:text_length]
    text_tensor, segment_tensor, positions_tensor = build_tensors_from_ids([ids] * beam_width, None, vocab, text_length + reserved_space + 2)
    return text_tensor, segment_tensor, positions_tensor, text_length

In [31]:
def make_tensors_for_sampling(tokens, vocab , reserved_space, max_text_length, beam_width=1):
    ids = transform_bpe_to_ids([tokens], vocab)[0]
    return make_tensors_for_sampling_ids(ids, vocab, reserved_space, max_text_length, beam_width)

In [32]:
def make_tensors_for_sampling_raw(text, bpe, vocab, reserved_space, max_text_length, beam_width=1):
    tokens = bpe.encode(text)
    return make_tensors_for_sampling(tokens, vocab, reserved_space, max_text_length, beam_width)

In [33]:
print(*make_tensors_for_sampling_raw('Погода в Москве на сегодня, точный прогноз погоды на сегодня для населенного пункта Москва,', 
                                     bpe_ru, vocab, 5, 5, 1), sep='\n')


tensor([[16358,     8,   257,    10,  1561,     2,     0,     0,     0,     0,
             0,     0]])
tensor([[4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5]])
tensor([[0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5]])
5


In [34]:
import pytorch_transformers

### Создаём сеть

In [35]:
class GPT2SummaryHead(nn.Module):
    def __init__(self, gpt_config, ignore_positions=False):
        super().__init__()
        self.gpt = pytorch_transformers.GPT2Model(gpt_config)
        self.ignore_positions = ignore_positions
        
    def encode(self, input_ids, segment_ids, position_ids):
        if self.ignore_positions:
            position_ids = None
        hidden_states = self.gpt(input_ids, position_ids=position_ids, token_type_ids=segment_ids, head_mask=None)[0]
        return hidden_states
    
    def compute_logits(self, decoder_states):
        return torch.matmul(decoder_states, self.gpt.wte.weight.t())
        
    def forward(self, input_ids, segment_ids, position_ids):
        return self.compute_logits(self.encode(input_ids, segment_ids, position_ids))

In [36]:
def sample_gpt(net, model_inputs, starting_index, vocab, n_times, top_k=None, device='cuda'):
    tensor_inputs, tensor_segments, tensor_positions = [t.to(device) for t in model_inputs]

    net.eval()
#     encoder_states = net.encoder(tensor_inputs, tensor_lengths)
    
    beam_width = tensor_inputs.size(0)
    results = [[] for _ in range(beam_width)]
    
    with torch.no_grad():
        for pos in range(starting_index, starting_index + n_times):
            hidden_states = net.encode(tensor_inputs, tensor_segments, tensor_positions)
            
            # B N V
            logits = net.compute_logits(hidden_states[:,pos:pos+1,:]).squeeze(1)
                        
            probs = F.softmax(logits, dim=-1)
            
            if top_k:
                probs, sampling_set = torch.topk(probs,top_k, dim=-1, sorted=False)
                
            cat = torch.distributions.Categorical(probs)
            if top_k:
                sampled = sampling_set[torch.arange(beam_width),cat.sample()]
            else:
                sampled = cat.sample()
                
            tensor_inputs[:,pos+1] = sampled
            
            for i, word_id in enumerate(tensor_inputs[:,pos+1].tolist()):
                results[i].append(vocab.id2word(word_id))
                
    return results

In [105]:
def sample_raw(net, text, bpe, vocab, n_times, max_text_length, beam_width=1, top_k=None, device='cuda'):
    tokens = encode_text(bpe, text, None, None)
    sample_tensor, sample_segments, sample_positions, starting_index = make_tensors_for_sampling(tokens,
                                                                                                 vocab, 
                                                                                                 reserved_space=n_times,
                                                                                                 max_text_length=max_text_length,
                                                                                                 beam_width=beam_width)
    
    return sample_gpt(net,(sample_tensor, sample_segments, sample_positions),
                      starting_index, vocab, n_times, top_k, device)

In [101]:
def sample_tokens(net, tokens, vocab, n_times, max_text_length, top_k=None, device='cuda'):
    sample_tensor, sample_segments, sample_positions, starting_index = make_tensors_for_sampling(tokens,
                                                                                                 vocab, 
                                                                                                 reserved_space=n_times,
                                                                                                 max_text_length=max_text_length,
                                                                                                 beam_width=beam_width)
    
    return sample_gpt(net,(sample_tensor, sample_segments, sample_positions),
                      starting_index, vocab, n_times, top_k, device)

In [102]:
def beam_search_tokens(net, tokens, vocab, n_times, max_text_length, beam_width,device='cuda',score='none'):
    sample_tensor, sample_segments, sample_positions, starting_index = make_tensors_for_sampling(tokens,
                                                                                                 vocab, 
                                                                                                 reserved_space=n_times,
                                                                                                 max_text_length=max_text_length,
                                                                                                 beam_width=beam_width)
    return beam_search_gpt(net, (sample_tensor, sample_segments, sample_positions), starting_index, vocab, n_times, device, score)

In [106]:
def beam_search_raw(net,  text, bpe, vocab, n_times, max_text_length, beam_width,device='cuda',score='none'):
    tokens = encode_text(bpe, text, None, None)
    return beam_search_tokens(net, tokens, vocab, n_times, max_text_length, beam_width, device, score)

In [41]:
def beam_search_gpt(net, model_inputs, starting_index, vocab, n_times, device='cuda', score='none'):
    tensor_inputs, tensor_segments, tensor_positions = [t.to(device) for t in model_inputs]

    net.eval()
#     encoder_states = net.encoder(tensor_inputs, tensor_lengths)
    
    beam_width = tensor_inputs.size(0)
    results = [[] for _ in range(beam_width)]
    log_likelihoods = torch.zeros(beam_width, dtype=torch.float32, device=device)
    
    scores = torch.zeros(beam_width, dtype=torch.float32, device=device)
    stopped = torch.zeros(beam_width, dtype=torch.int8, device=device)
    eos_index = vocab.word2id(EOS_TOKEN)
    with torch.no_grad():
        for pos in range(starting_index, starting_index + n_times):
            if pos == starting_index:
                hidden_states = net.encode(tensor_inputs[0:1], tensor_segments[0:1], tensor_positions[0:1])
                logits = net.compute_logits(hidden_states[:,pos:pos+1,:]).view(-1)
                log_likelihoods, indices = torch.topk(F.log_softmax(logits, dim=0), beam_width)
                tensor_inputs[:,pos+1] = indices
                stopped[indices == eos_index] = 1
            else:
                hidden_states = net.encode(tensor_inputs, tensor_segments, tensor_positions)

                # B N V
                logits = net.compute_logits(hidden_states[:,pos:pos+1,:]).squeeze(1)
                # B V
                log_probs = F.log_softmax(logits, dim=-1)

                new_log_likelihoods = log_likelihoods.view(-1,1) + log_probs
                
                if score == 'divide':
                    new_scores = new_log_likelihoods / (pos - starting_index+1)
                else:
                    new_scores = new_log_likelihoods
                    
                new_scores[stopped==1] = -1e9 
                new_scores[stopped==1, eos_index] = scores[stopped==1]
                new_log_likelihoods[stopped==1] = -1e9
                new_log_likelihoods[stopped==1, eos_index] = log_likelihoods[stopped==1]
                
                top_scores, top_indices = torch.topk(new_scores.view(-1), beam_width)
                
                ancestor_idx, successor_word_idx = np.unravel_index(top_indices.tolist(), shape=tuple(logits.size()))

                new_tensor_inputs = torch.empty(*tensor_inputs.size(), device=device, dtype=tensor_inputs.dtype)
                new_stopped = torch.zeros(beam_width, dtype=torch.int8, device=device)
                
                for i, (ancestor, successor) in enumerate(zip(ancestor_idx, successor_word_idx)):
#                     print(ancestor, successor)
                    new_tensor_inputs[i] = tensor_inputs[ancestor]
                    new_tensor_inputs[i,pos+1] = int(successor)
                    if stopped[ancestor].item() or int(successor) == eos_index:
                        new_stopped[i] = 1
                        new_tensor_inputs[i,pos+1] = eos_index

                tensor_inputs = new_tensor_inputs
                stopped = new_stopped
                log_likelihoods = new_log_likelihoods.view(-1)[top_indices]
                scores = top_scores
            

    for pos in range(starting_index, starting_index + n_times):
        for i, word_id in enumerate(tensor_inputs[:,pos+1].tolist()):
            results[i].append(vocab.id2word(word_id))
                
    return results, scores.tolist(), log_likelihoods.tolist()

In [42]:
def print_samples(samples):
    for s in samples:
        print('|'.join(s))
        print('---------------------')

In [43]:
decoder_config = pytorch_transformers.GPT2Config(vocab_size_or_config_json_file=len(vocab),
                                                 n_special=len(special_symbols),
                                                 n_positions=350,
                                                  n_ctx=400,
                                                  n_embd=300, 
                                                  n_layer=5,
                                                  n_head=4,
                                                  embd_pdrop=0.3
                                                 )

In [44]:
decoder_model =  GPT2SummaryHead(decoder_config)
decoder_model.gpt.wte.weight.data[vocab.n_specials:].copy_(torch.from_numpy(pretrained_embeddings[vocab.n_specials:]))
None

In [45]:
sample_text1 = 'Погода в Москве на сегодня, точный прогноз погоды на сегодня для населенного пункта Москва'

In [46]:
print_samples(sample_raw(decoder_model, sample_text1, bpe_ru, vocab, 20, 100, beam_width=5, top_k=10, device='cpu'))


%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"
---------------------
%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"
---------------------
▁sms|▁sms|▁sms|▁sms|▁sms|▁sms|▁sms|▁sms|▁sms|▁sms|▁sms|▁sms|▁sms|▁sms|▁sms|▁sms|▁sms|▁sms|▁sms|▁sms
---------------------
▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный
---------------------
%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"
---------------------


In [47]:
print_samples(beam_search_raw(decoder_model, sample_text1, bpe_ru, vocab, 20, 100, beam_width=5, device='cpu')[0])


%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"|%"
---------------------
▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный|▁истребительный
---------------------
вказ|вказ|вказ|вказ|вказ|вказ|вказ|вказ|вказ|вказ|вказ|вказ|вказ|вказ|вказ|вказ|вказ|вказ|вказ|вказ
---------------------
▁width|▁width|▁width|▁width|▁width|▁width|▁width|▁width|▁width|▁width|▁width|▁width|▁width|▁width|▁width|▁width|▁width|▁width|▁width|▁width
---------------------
округ|округ|округ|округ|округ|округ|округ|округ|округ|округ|округ|округ|округ|округ|округ|округ|округ|округ|округ|округ
---------------------


In [48]:
def masked_cross_entropy(logits, targets, mask):
    losses = F.cross_entropy(logits, targets, reduction='none')
    return losses[mask.view(-1)].sum() / mask.sum()

In [49]:
def compute_lm_loss(context, batch_state):
    input_ids, input_segments, input_positions = (x.to(context.device) for x in batch_state.batch)
    decoder_input_ids = input_ids[:, :-1]
    decoder_input_segments = input_segments[:, :-1]
    decoder_input_positions = input_positions[:, :-1]
    target = input_ids[:, 1:]
    logits = context.model(decoder_input_ids, decoder_input_segments, decoder_input_positions)
    loss_mask = (target != 0) & (target != ID_SEP)
    
    return masked_cross_entropy(logits.view(-1, logits.size(-1)), target.contiguous().view(-1), loss_mask)

In [50]:
def compute_summary_loss(context, batch_state):
    input_ids, input_segments, input_positions = (x.to(context.device) for x in batch_state.batch)
    decoder_input_ids = input_ids[:, :-1]
    decoder_input_segments = input_segments[:, :-1]
    decoder_input_positions = input_positions[:, :-1]
    target = input_ids[:, 1:]
    logits = context.model(decoder_input_ids, decoder_input_segments, decoder_input_positions)
    
    loss_mask = (target != 0) & (decoder_input_segments == ID_SEGMENT_SUMMARY)
    
    return masked_cross_entropy(logits.view(-1, logits.size(-1)), target.contiguous().view(-1), loss_mask)

In [51]:
del decoder_model

In [52]:
gpt_config = pytorch_transformers.GPT2Config(vocab_size_or_config_json_file=len(vocab),
                                                 n_special=len(special_symbols),
                                                 n_positions=350,
                                                  n_ctx=400,
                                                  n_embd=300, 
                                                  n_layer=5,
                                                  n_head=4,
                                                  embd_pdrop=0.3
                                                 )

In [53]:
gpt_model =  GPT2SummaryHead(decoder_config)
gpt_model.gpt.wte.weight.data[vocab.n_specials:].copy_(torch.from_numpy(pretrained_embeddings[vocab.n_specials:]))
None

In [54]:
lr = 6e-4
max_grad_norm = 10.0
num_total_steps = 113000
num_warmup_steps = 5000
train_batch_size = 16
gradient_accumulation_steps = 2
n_epoch=20

optimizer = pytorch_transformers.AdamW(gpt_model.parameters(), lr=lr, correct_bias=False)
scheduler = pytorch_transformers.WarmupLinearSchedule(optimizer, warmup_steps=num_warmup_steps, t_total=num_total_steps)

In [55]:
gpt_model = gpt_model.cuda()

In [56]:
def current_timestamp():
    return datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S')

In [66]:
def sample_after_epoch(model, idx=None):
    idx = idx if idx is not None else np.random.randint(0, len(val_raw_texts)) # 9217
    print(idx, val_raw_texts[idx])

    tokens = val_texts[idx]
    sample_tensor, sample_segments, sample_positions, starting_index = make_tensors_for_sampling(tokens, vocab, reserved_space=20, max_text_length=220,beam_width=5)

    sr = sample_gpt(model, (sample_tensor, sample_segments, sample_positions), starting_index, vocab, 20, top_k=15, device='cuda')
    print('\nRandom samples: ')
    print_samples(sr)
    print('\n\nBeam search samples:')
    
    sr, scores, lls = beam_search_gpt(model, (sample_tensor, sample_segments, sample_positions), starting_index, vocab, 20, device='cuda',score='divide')
    
    print_samples(sr)
    print(scores)
    print(lls)

In [72]:
sample_after_epoch(gpt_model)

11499 В Луизиане буксирующее судно врезалось в основание заброшенной нефтеплатформы, спровоцировав утечку нефти, передает 27 июля Associated Press. Инцидент произошел в акватории озера Мад (Mud Lake), расположенного в заливе Баратария, который в свою очередь входит в состав обширного Мексиканского залива. По сообщениям береговых служб, буксир при транспортировке другого судна задел устьевое оборудование скважины, в результате чего оттуда начала разливаться нефть. Объемы разливающейся нефти пока не установлены. Отмечается, что струя нефти и газа, бьющая из поврежденной скважины, достигает 30 метров в высоту. На поверхности озера уже сформировалась нефтяная пленка протяженностью в два километра. Как подчеркивают представители экстренных служб, в месте ЧП уже работают суда-нефтесборщики. Ожидается, что поврежденная нефтескважина будет закрыта в ближайшее время. По заявлениям пресс-службы губернатора Бобби Джиндала (Bobby Jindal), скважина, о которой идет речь, принадлежала хьюстонской ком


Random samples: 
▁в|▁луизи|ане|▁букси|р|▁столк|нулась|▁с|▁нефтя|ной|▁уте|чкой|</S>|▁нефть|</S>|▁нефть|</S>|</S>|▁нефть|</S>
---------------------
▁в|▁зато|ну|вшем|▁в|▁луизи|ане|▁букси|ру|▁вреза|лось|▁в|▁стену|▁нефте|плат|формы|</S>|</S>|</S>|</S>
---------------------
▁в|▁луизи|ане|▁зато|нуло|▁судно|</S>|▁нефть|</S>|▁нефть|</S>|</S>|</S>|▁катастрофа|</S>|▁нефть|</S>|▁нефть|</S>|▁нефть
---------------------
▁во|▁второй|▁раз|▁за|стря|вший|▁в|▁пото|плении|▁нефть|▁танк|▁вреза|лась|▁в|▁залив|</S>|00|▁метров|</S>|</S>
---------------------
▁в|▁луизи|ане|▁букси|р|▁вреза|лся|▁в|▁пристань|</S>|0|</S>|0|</S>|</S>|</S>|▁прекращены|</S>|</S>|</S>
---------------------


Beam search samples:
▁в|▁луизи|ане|▁букси|р|▁врезался|▁в|▁здание|▁нефте|плат|формы|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁в|▁луизи|ане|▁букси|ра|▁вреза|лась|▁в|▁здание|▁нефте|плат|формы|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁в|▁луизи|ане|▁букси|р|▁врезался|▁в|▁воду|</S>|</S>|</S

In [ ]:
checkpoint = Checkpoint('gpt_checkpoints/chk{}_epoch{}_loss{:.3f}')
checkpoint.loss = 15

In [87]:
print(checkpoint.best_path)

None


In [70]:

def after_epoch(context, epoch_state):
    loss = epoch_state.val_loss
    sample_after_epoch(context.model)
    if loss < checkpoint.loss:
        print("Saving checkpoint")
        checkpoint.update({'config': gpt_config, 
                           'model': gpt_model.state_dict(), 
                           'optimizer': optimizer.state_dict(), 
                           'scheduler': scheduler.state_dict()},
                          index=(current_timestamp(), epoch_state.epoch+1,loss), best=True)
        checkpoint.loss = loss 

In [59]:
after_epoch(general_training.AttrDict({'model': gpt_model}),general_training.AttrDict({'epoch': 4, 'val_loss': 4.145}))

9168 Американская компания Ad Astra провела первые испытания плазменного двигателя на сверхпроводящих магнитах VX-200. Об этом сообщается в пресс-релизе компании. Следующие испытания двигателя запланированы на 14 июля 2009 года. Предыдущие испытания, которые проводились осенью 2008 года, позволили доказать работоспособность двигателя. Тогда в нем был установлен обычный магнит. Использование сверхпроводящего аналога позволило увеличить мощность VX-200 примерно в 10 раз. Видео испытания доступно здесь. Испытанный двигатель относится к системе VASIMR (Variable Specific Impulse Magnetoplasma Rocket) - магнитоплазменных ракетных двигателей с переменным импульсом. Ранее подобные двигатели разрабатывались NASA, однако в настоящее время исследования полностью ведет компания Ad Astra, расположенная в Коста-Рике. Принцип работы двигателя заключается в следующем: в специальной камере под воздействием электромагнитных волн материя (обычно благородный газ) ионизируется. Под воздействием магнитного 



Random samples: 
▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной|▁архитектурной
---------------------
▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза|▁бронза
---------------------
▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной
---------------------
▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|▁виртуальной|

In [60]:
def after_step(context, batch_state):
#     current_lr = next(iter(context.optimizer.param_groups))['lr']
#     writer.add_scalar('Learning rate', current_lr, batch_state.total_steps)
#     writer.add_scalar('Training loss', batch_state.loss, batch_state.total_steps)
    scheduler.step()

In [61]:
def clip_grad(context, batch_state):
    torch.nn.utils.clip_grad_norm_(context.model.parameters(), max_grad_norm)

In [62]:
train_dataset = TensorDataset(train_text_tensor, train_segment_tensor, train_positions_tensor)
val_dataset = TensorDataset(val_text_tensor, val_segment_tensor, val_positions_tensor)

In [63]:
# train_dataset = TensorDataset(train_text_tensor[:100], train_segment_tensor[:100], train_positions_tensor[:100])
# val_dataset = TensorDataset(val_text_tensor[:100], val_segment_tensor[:100], val_positions_tensor[:100])

In [64]:
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [75]:
checkpoint.best_path

In [264]:
general_training.train_general(gpt_model, optimizer,
                              compute_lm_loss, compute_summary_loss,
                              train_loader, val_loader, 
                              n_epoch, gradient_accumulation_steps=gradient_accumulation_steps,
                              n_prints=4, after_epoch=after_epoch, after_gradient=clip_grad, after_step=after_step)
None

In [77]:
persisted = torch.load('gpt_checkpoints/chk2019-08-26_065533_epoch16_loss2.221')

In [79]:
print(persisted.keys())

dict_keys(['config', 'model', 'optimizer', 'scheduler'])


In [80]:
gpt_model.load_state_dict(persisted['model'])

<All keys matched successfully>

In [82]:
optimizer.load_state_dict(persisted['optimizer'])
scheduler.load_state_dict(persisted['scheduler'])

In [86]:
!nvidia-smi

Mon Aug 26 07:15:06 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 25%   46C    P2    57W / 250W |   1701MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [85]:
torch.cuda.empty_cache()

In [88]:
importlib.reload(general_training)

<module 'utils.general_training' from '/home/lenta/utils/general_training.py'>

In [89]:
general_training.train_general(gpt_model, optimizer,
                              compute_lm_loss, compute_summary_loss,
                              train_loader, val_loader, 
                              4, gradient_accumulation_steps=gradient_accumulation_steps,
                              n_prints=4, initial_epoch=16, after_epoch=after_epoch, after_gradient=clip_grad, after_step=after_step)

Epoch 17 Iteration 2812 Loss 1.6573249956706848
Epoch 17 Iteration 5624 Loss 1.658568224603387
Epoch 17 Iteration 8436 Loss 1.6585102523034532
Epoch 17 Iteration 11248 Loss 1.6589942226559984


Epoch 17 val_Loss 2.2140766849517823
11235 Свидетель нападения на актера Александра Ляпина, известного по сериалу «Интерны», рассказал о произошедшем в одном из московских кафе. Как он уточнил телеканалу НТВ, артист находился в состоянии алкогольного опьянения. По словам свидетеля, охранникам заведения не понравилось, что артист был пьян, они вывели его на улицу и «потолкались». «Вроде побоев, драки не было, просто потолкались, и потом подъехала полиция», — рассказал он. По информации Life, в потасовке участвовали посетители, которые также были в нетрезвом состоянии. Портал отмечает, что тяжелых травм, о которых сообщалось ранее, актер не получал. Участники конфликта были задержаны. По факту произошедшего инициирована проверка. Инцидент произошел в ночь на 27 июня возле ночного клуба на Грузинском Валу. Как сообщалось, на молодого человека и его подругу Полину Левашкевич напали семь человек. В результате актер получил сотрясение мозга и травму носа. 30-летний Ляпин снимается в кино и с


Random samples: 
▁в|▁московском|▁ресторане|▁устроил|▁нападение|▁на|▁актера|▁на|▁актера|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁сми|▁рассказа|ли|▁о|▁нападении|▁на|▁актера|▁алексея|▁ля|пина|</S>|х|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁напа|вший|▁на|▁актера|▁ля|пина|▁из|▁ресторана|▁на|▁улице|▁ля|пина|▁рассказал|▁о|▁произошед|шем|</S>|</S>|</S>|</S>
---------------------
▁в|▁москве|▁у|▁пья|ных|▁посетителей|▁напа|вшего|▁на|▁актера|▁напали|▁в|▁кафе|▁с|▁алкого|льными|▁оп|ья|нения|ми
---------------------
▁ск|▁рассказал|▁о|▁поку|шении|▁на|▁актера|▁в|▁ресторане|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------


Beam search samples:
▁свидетель|▁нападения|▁на|▁актера|▁ля|пина|▁рассказал|▁о|▁произошед|шем|▁в|▁кафе|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁свидетель|▁нападения|▁на|▁актера|▁ля|пина|▁рассказал|▁о|▁произошед|шем|▁в|▁кафе|▁«|интер|ны|»|</S>|</S>|</S>|</S>
-------------------

Epoch 18 Iteration 2812 Loss 1.6522933844143093
Epoch 18 Iteration 5624 Loss 1.6526100246784188
Epoch 18 Iteration 8436 Loss 1.6528366976238755
Epoch 18 Iteration 11248 Loss 1.652500656143276


Epoch 18 val_Loss 2.206396284421285
11042 К обвиняемой в смерти блокадницы после конфликта в петербургском магазине «Магнит» применили амнистию. Об этом сообщает ТАСС из зала Кронштадтского районного суда. «Суд считает необходимым удовлетворить ходатайство [бывшего директора магазина Ольги] Конюховой об амнистии в соответствии с постановлением Госдумы об амнистии к 70-летию Победы за преступления, предусматривающие наказание до 5 лет лишения свободы», — заявила судья. Она отметила, что Конюхова обвиняется в преступлениях небольшой тяжести, а обстоятельств, препятствующих применению амнистии, не установлено. В связи с этим суд прекращает уголовное дело. Ходатайство было заявлено в четверг, 18 июня. Адвокат, представляющий интересы родственников 81-летней блокадницы Раузы Галимовой, отметил, что основания для применения акта амнистии есть. Племянница Галимовой Дина Соколова просила суд провести разбирательство. «Я хочу, чтобы она была наказана, а потом получила амнистию», — отметила она.

...

Epoch 19 val_Loss 2.2012045011520387
8059 Шесть израильских солдат были ранены в перестрелках с партизанами "Хизбалла", произошедших в воскресенье на севере Израиля. Первый инцидент произошел у поста Армии обороны Израиля в районе Хар-Дов (Har Dov). Двое солдат получили ранения легкой и средней степеней тяжести. В перестрелке боевики "Хизбаллы" применили минометы и противотанковые ракеты. Спустя некоторое время еще четыре женщины-военнослужащих были ранены в ходе минометного обстрела военного аванпоста в Мошав-Авивим (Moshav Avivim), в районе верхней Галилеи (Upper Galilee). В обоих случаях израильская сторона отвечала артиллерийским огнем. В районы столкновений также были направлены военные вертолеты. Боевики "Хизбаллы" мотивируют свои действия тем, что граница между Ливаном и Израилем неверна и район Хар-Дов должен принадлежать Ливану. Сам Ливан признает границу с Израилем законной. Министр обороны страны Халил аль-Храви (Khalil al-Hrawi) заявил в воскресенье: "Позиция ливанского пра


Random samples: 
▁израиль|ские|▁солдаты|▁обстре|ляли|▁из|▁грана|то|метов|</S>|х|</S>|х|</S>|ха|</S>|</S>|ха|ви|м
---------------------
▁израиль|ские|▁солдаты|▁расстреляли|▁израильского|▁солдата|</S>|▁армия|</S>|льская|</S>|▁армия|</S>|▁армия|</S>|лена|</S>|хия|</S>|▁не
---------------------
▁израиль|ские|▁солдаты|▁ранены|▁в|▁перестре|лке|▁со|▁стороны|▁сил|▁обороны|▁израиля|</S>|▁ранены|</S>|</S>|хия|</S>|хия|</S>
---------------------
▁израильский|▁солдат|▁обстре|лял|▁солдат|▁в|▁секторе|▁газа|</S>|лах|</S>|▁армия|▁обороны|</S>|та|</S>|хия|</S>|</S>|хия
---------------------
▁четыре|▁палестин|ца|▁ранены|▁в|▁ответ|▁на|▁военные|▁действия|▁израиля|</S>|▁армия|</S>|▁армия|</S>|▁армия|</S>|</S>|хия|</S>
---------------------


Beam search samples:
▁в|▁перестре|лках|▁с|▁"|хи|з|бал|лой|"|▁ранены|▁два|▁израильских|▁солдата|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁в|▁перестре|лках|▁с|▁"|хи|з|бал|лой|"|▁погибли|▁два|▁израильских|▁солдата|</S>|</S>|</S>|</S>|</S>|</S>
----------------

Epoch 20 Iteration 2812 Loss 1.6423811360578957
Epoch 20 Iteration 5624 Loss 1.6417035261012751
Epoch 20 Iteration 8436 Loss 1.6419111271642746
Epoch 20 Iteration 11248 Loss 1.6419290392679475


Epoch 20 val_Loss 2.1952906808853148
1251 Южная Корея нанесла высокоточные ракетные удары по учебным целям в ответ на запуск баллистической ракеты КНДР. Об этом во вторник, 28 ноября, сообщает ТАСС со ссылкой на заявление Комитета начальника штабов вооруженных сил страны. Глава комитета Госдумы по обороне Юрий Швыткин сообщил «Интерфаксу», что запуск ракеты КНДР ставит под сомнение безопасность делегации российского парламента, которая сейчас находится там с официальным визитом. «Могут последовать ответные действия со стороны Южной Кореи, со стороны Японии, США», — пояснил он. Ранее во вторник южнокорейские военные заявили о ракетном пуске с территории Северной Кореи. Отмечалось, что он был осуществлен в провинции Пхеннан-Намдо. Ракета полетела в восточном направлении.



Random samples: 
▁южная|▁корея|▁нанесла|▁удар|▁по|▁ракетной|▁ракетной|▁цели|</S>|там|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁кндр|▁нанесла|▁ракетные|▁удары|▁по|▁объектам|▁северной|▁кореи|</S>|0|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁кндр|▁произвела|▁балли|стическую|▁атаку|▁кндр|</S>|сша|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|▁сша|</S>|</S>|</S>
---------------------
▁кндр|▁пора|зила|▁ядерный|▁запуск|▁ракет|у|▁кндр|</S>|</S>|тах|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁южная|▁корея|▁произвела|▁ответный|▁запуск|▁балли|стической|▁ракеты|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------


Beam search samples:
▁кндр|▁нанесла|▁ракетные|▁удары|▁по|▁учебным|▁целям|▁в|▁ответ|▁на|▁запуск|▁кндр|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁кндр|▁нанесла|▁ракетные|▁удары|▁по|▁учебным|▁целям|▁в|▁ответ|▁на|▁запуск|▁северокорей|ской|▁ракеты|</S>|</S>|</S>|</

In [110]:
sample_text2 = """Российский космонавт Александр Скворцов, выполнявший в ручном режиме перестыковку корабля "Союз МС-13" с одного модуля МКС на другой, использовал минимально возможное количество топлива, сообщил журналистам руководитель полета российского сегмента Международной космической станции Владимир Соловьев журналистам.

Он отметил, что примерно в 07:30 по московскому времени на корабле "Союз МС-14" с роботом FEDOR будет проведен двухимпульсный маневр с целью выстраивания оптимальной траектории полёта при стыковке корабля с МКС, которая намечена на утро вторника.

В понедельник утром экипаж "Союза МС-13", на борту которого находились командир корабля Александр Скворцов и два бортинженера Лука Пармитано и Эндрю Морган, отстыковался от причального модуля "Звезда", а затем в ручном режиме был пристыкован к МИМ2.

Во вторник к модулю "Звезда" должен пристыковаться корабль "Союз МС-14" с человекоподобным роботом FEDOR. До этого, в понедельник, в 08:30 этот корабль также совершит специальный маневр, чтобы во вторник начать вторую попытку стыковки с МКС. Она намечена на 6:12 вторника.

В субботу транспортный пилотируемый корабль "Союз МС-14" не смог пристыковаться к МКС. По предварительным данным, причиной неудачи стали проблемы с радиооборудованием, задействованном на ближнем участке приближения корабля к станции во время автоматической стыковки при помощи системы "Курс"."""

In [246]:
sample_text3 = """
Космический аппарат (КА) «Глонасс-М» № 742 российской глобальной навигационной спутниковой системы ГЛОНАСС, запущенный в октябре 2011 года, выведен 26 августа на внеплановое техническое обслуживание, о котором заранее не говорилось. Об этом сообщает РИА Новости.

Агентство обратило внимание на данные информационно-аналитического центра ГЛОНАСС, согласно которым в настоящее время 22 КА используются по целевому назначению, а два КА временно выведены на техобслуживание.

Ранее в августе агентство упомянуло, что более половины КА системы ГЛОНАСС работают за пределами гарантийного срока. 1 августа старейший КА ГЛОНАСС («Глонасс-М» № 717) был выведен на незапланированное техобслуживание.

Для покрытия территории всей Земли ГЛОНАСС необходима активная работа 24 КА, тогда как для покрытия всей России достаточно 18 активных КА.

В июне в заключении Счетной палаты по проекту поправок к федеральному бюджету на 2019 год указывалось, что санкции стран Запада, введенные в отношении электроники военного и двойного назначений, привели к сокращению финансирования ГЛОНАСС. В апреле замгендиректора «Роскосмоса» Юрий Урличич заявил, что точность ГЛОНАСС к 2025 году повысится на четверть. В том же месяце источники сообщали, что завершение эксплуатации тяжелых ракет «Протон-М» и неготовность создаваемых им на замену носителей «Ангара-А5» привели к необходимости уменьшения массы спутников ГЛОНАСС.

В марте президент некоммерческого партнерства системы Александр Гурко заявил, что Индия будет производить навигационные чипсеты (составная часть навигационных систем для получения сигналов со спутников) ГЛОНАСС российской разработки. В апреле 2018-го гендиректор ИСС Николай Тестоедов рассказал, что космические аппараты ГЛОНАСС почти на 40 процентов состоят из зарубежных комплектующих.
"""

In [255]:
print(encode_text(bpe_ru,sample_text2, None, None)[:260])

['▁российский', '▁космонав', 'т', '▁александр', '▁сквор', 'цов', ',', '▁выполня', 'вший', '▁в', '▁ру', 'чном', '▁режиме', '▁пере', 'сты', 'ков', 'ку', '▁корабля', '▁"', 'союз', '▁мс', '-00', '"', '▁с', '▁одного', '▁модуля', '▁мкс', '▁на', '▁другой', ',', '▁использовал', '▁минима', 'льно', '▁возможное', '▁количество', '▁топлива', ',', '▁сообщил', '▁журналиста', 'м', '▁руководитель', '▁полета', '▁российского', '▁сегмента', '▁международной', '▁космической', '▁станции', '▁владимир', '▁солов', 'ьев', '▁журналиста', 'м', '.', '▁он', '▁отметил', ',', '▁что', '▁примерно', '▁в', '▁00:00', '▁по', '▁московскому', '▁времени', '▁на', '▁корабле', '▁"', 'союз', '▁мс', '-00', '"', '▁с', '▁робо', 'том', '▁fed', 'or', '▁будет', '▁проведен', '▁дву', 'хим', 'пуль', 'сный', '▁манев', 'р', '▁с', '▁целью', '▁вы', 'страи', 'вания', '▁оптима', 'льной', '▁траектории', '▁полёта', '▁при', '▁сты', 'ков', 'ке', '▁корабля', '▁с', '▁мкс', ',', '▁которая', '▁наме', 'чена', '▁на', '▁утро', '▁втор', 'ника', '.', '▁в', '

In [261]:
sr, scores, lls = beam_search_raw(gpt_model, sample_text2, bpe_ru, vocab, 22, 260, 40, score='none')

In [262]:
print_samples(sr)

▁экипаж|▁"|союза|▁мс|-00|"|▁при|сты|кова|лся|▁к|▁мкс|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁"|союз|▁мс|-00|"|▁от|сты|кова|лся|▁от|▁мкс|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁экипаж|▁"|союза|▁мс|-00|"|▁от|сты|кова|лся|▁от|▁мкс|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁космонав|т|▁"|союз|▁мс|-00|"|▁от|сты|кова|лся|▁от|▁мкс|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁российский|▁космонав|т|▁"|союз|▁мс|-00|"|▁от|сты|кова|лся|▁от|▁мкс|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁экипаж|▁"|союза|▁мс|-00|"|▁получил|▁минима|льную|▁массу|▁топлива|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁экипаж|▁"|союза|▁мс|-00|"|▁при|сты|кова|лся|▁к|▁мкс|▁к|▁мкс|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>
---------------------
▁экипаж|▁"|союза|▁мс|-00|"|▁сделал|▁оптима|льную|▁посадку|▁на|▁мкс|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S>|</S

In [263]:
for s in sr:
    print(bpe_ru.decode(s))

экипаж "союза мс-00" пристыковался к мкс</S></S></S></S></S></S></S></S></S></S>
"союз мс-00" отстыковался от мкс</S></S></S></S></S></S></S></S></S></S></S>
экипаж "союза мс-00" отстыковался от мкс</S></S></S></S></S></S></S></S></S></S>
космонавт "союз мс-00" отстыковался от мкс</S></S></S></S></S></S></S></S></S>
российский космонавт "союз мс-00" отстыковался от мкс</S></S></S></S></S></S></S></S>
экипаж "союза мс-00" получил минимальную массу топлива</S></S></S></S></S></S></S></S></S></S></S>
экипаж "союза мс-00" пристыковался к мкс к мкс</S></S></S></S></S></S></S></S>
экипаж "союза мс-00" сделал оптимальную посадку на мкс</S></S></S></S></S></S></S></S></S></S>
экипаж "союза мс-00" нашел оптимальное количество топлива</S></S></S></S></S></S></S></S></S></S></S>
космонавт "союз мс-00" получил минимальные двигатели</S></S></S></S></S></S></S></S></S></S></S>
экипаж "союза мс-00" застряковался на мкс</S></S></S></S></S></S></S></S></S></S>
экипаж "союза мс-00" пристыковался на мкс<